In [2]:
%reset -f

In [3]:
#.\.venv\Scripts\Activate.ps1

import pandas as pd
import numpy as np
from pathlib import Path

bases_path = Path("bases")

checklist = bases_path / "HERING_FRANQUIA_checklist_RA_NORMAL_RA_2025-12-10.xlsx"
relatorio = bases_path / "relatorio_ra-todas_03e04-12-2025.xlsx"
estoque = bases_path / "STK_QLIK_10.xlsx"
carteira = bases_path / "CA_APS_2.0.xlsx"
carteira_fat = bases_path / "CARTEIRA_F_2.0.xlsx"
df_check = pd.read_excel(checklist, usecols=['FILIAL', 'ARTIGO_COR', 'PRESENTE', 'TRANSITO', 'EMBALADO', 'RESERVADO'], engine="calamine", sheet_name="Tabela1")
df_relatorio = pd.read_excel(relatorio, header=5, usecols=['Cod. Loja', 'Artigo', 'Cor', 'Atendido (SAP)', 'Cancelada'], engine="calamine")
df_estoque = pd.read_excel(estoque, engine="calamine")
df_cat = pd.read_excel(carteira, engine="calamine")
df_cat_fat_hoje_ontem = pd.read_excel(carteira_fat, engine="calamine")

In [4]:
df_check.columns

Index(['FILIAL', 'ARTIGO_COR', 'PRESENTE', 'TRANSITO', 'EMBALADO',
       'RESERVADO'],
      dtype='object')

In [5]:
df_check.dtypes

FILIAL         int64
ARTIGO_COR    object
PRESENTE       int64
TRANSITO       int64
EMBALADO       int64
RESERVADO      int64
dtype: object

In [6]:
df_check['ARTIGO_COR'] = df_check['ARTIGO_COR'].str.replace('-', '')
df_check['FILIAL'] = df_check['FILIAL'].astype(str)
df_check['chave'] =  df_check['FILIAL'] + df_check['ARTIGO_COR'] 

In [7]:
df_check.head()
pd.reset_option('display.max_rows')

In [8]:
df_estoque.head()

,Dia,Canal,Ponto Venda Cód,Artigo,Pirâmide Atual,Peças,Estoque Total,Estoque em Trânsito
0,9,Franquias,59,02QYM2H07S,Sazonal,0,4,0
1,9,Franquias,59,02QYN0A00S,Sazonal,0,11,0
2,9,Franquias,59,02QYN1007S,Sazonal,0,2,0
3,9,Franquias,59,02TQAX7EN,Sazonal,1,32,0
4,9,Franquias,59,02TQEACEN,Sazonal,0,1,0


In [9]:
df_estoque.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5879 entries, 0 to 5878
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Dia                   5879 non-null   int64 
 1   Canal                 5879 non-null   object
 2   Ponto Venda Cód       5879 non-null   int64 
 3   Artigo                5879 non-null   object
 4   Pirâmide Atual        5879 non-null   object
 5   Peças                 5879 non-null   int64 
 6   Estoque Total         5879 non-null   int64 
 7   Estoque em Trânsito   5879 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 367.6+ KB


In [10]:
df_estoque.columns

Index(['Dia ', 'Canal ', 'Ponto Venda Cód ', 'Artigo ', 'Pirâmide Atual ',
       'Peças ', 'Estoque Total ', 'Estoque em Trânsito '],
      dtype='object')

In [11]:
df_estoque.columns = df_estoque.columns.str.strip()
colunas_numericas = ['Peças', 'Estoque Total', 'Estoque em Trânsito']
colunas_categoricas = ['Canal', 'Ponto Venda Cód', 'Pirâmide Atual']
df_estoque[colunas_numericas] = df_estoque[colunas_numericas].astype('Int64')
df_estoque[colunas_categoricas] = df_estoque[colunas_categoricas].astype('str')
print("\n\n---- Depois da limpeza ---")
df_estoque.dtypes



---- Depois da limpeza ---


Dia                     int64
Canal                  object
Ponto Venda Cód        object
Artigo                 object
Pirâmide Atual         object
Peças                   Int64
Estoque Total           Int64
Estoque em Trânsito     Int64
dtype: object

In [12]:
df_estoque.columns

Index(['Dia', 'Canal', 'Ponto Venda Cód', 'Artigo', 'Pirâmide Atual', 'Peças',
       'Estoque Total', 'Estoque em Trânsito'],
      dtype='object')

In [13]:
df_estoque['chave'] =  df_estoque['Ponto Venda Cód'] + df_estoque['Artigo'] 

In [14]:
df_estoque.head()

,Dia,Canal,Ponto Venda Cód,Artigo,Pirâmide Atual,Peças,Estoque Total,Estoque em Trânsito,chave
0,9,Franquias,59,02QYM2H07S,Sazonal,0,4,0,5902QYM2H07S
1,9,Franquias,59,02QYN0A00S,Sazonal,0,11,0,5902QYN0A00S
2,9,Franquias,59,02QYN1007S,Sazonal,0,2,0,5902QYN1007S
3,9,Franquias,59,02TQAX7EN,Sazonal,1,32,0,5902TQAX7EN
4,9,Franquias,59,02TQEACEN,Sazonal,0,1,0,5902TQEACEN


In [15]:
df_estoque['STK_REAL'] = df_estoque['Estoque Total'] - df_estoque['Peças']

In [16]:
df_estoque.head()

,Dia,Canal,Ponto Venda Cód,Artigo,Pirâmide Atual,Peças,Estoque Total,Estoque em Trânsito,chave,STK_REAL
0,9,Franquias,59,02QYM2H07S,Sazonal,0,4,0,5902QYM2H07S,4
1,9,Franquias,59,02QYN0A00S,Sazonal,0,11,0,5902QYN0A00S,11
2,9,Franquias,59,02QYN1007S,Sazonal,0,2,0,5902QYN1007S,2
3,9,Franquias,59,02TQAX7EN,Sazonal,1,32,0,5902TQAX7EN,31
4,9,Franquias,59,02TQEACEN,Sazonal,0,1,0,5902TQEACEN,1


In [17]:
df_estoque_agrupado = df_estoque.groupby('chave')['STK_REAL'].sum().reset_index()
df_estoque_agrupado
total = df_estoque_agrupado['STK_REAL'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_estoque_agrupado)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
               chave  STK_REAL
0      12910111AX7EN        16
1      12910111AZ2EN         3
2     12910111M2H07S        48
3     12910111MD307S        19
4     12910111N0A00S         8
...              ...       ...
5874    978SN81N10SN         1
5875    978SN97N10SN         3
5876    978SNB2N10SN         1
5877    978SNB3N10SN         1
5878    978SNCVA8ESN         1

[5879 rows x 2 columns]

 O total de estoque real no BI é: 85653


In [18]:
TD_STK =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])['PRESENTE'].sum().reset_index()
TD_STK = TD_STK.merge(df_estoque[['chave', 'STK_REAL']], how='left', on='chave' )
TD_STK['STK_REAL'] = TD_STK['STK_REAL'].fillna(0).astype(np.int64)
TD_STK['DIF_ABS'] = (TD_STK['PRESENTE'] - TD_STK['STK_REAL']).abs()
TD_STK

,chave,FILIAL,ARTIGO_COR,PRESENTE,STK_REAL,DIF_ABS
0,12910201AX7EN,1291,0201AX7EN,69,69,0
1,12910201HMJEN,1291,0201HMJEN,24,22,2
2,12910201M2H07S,1291,0201M2H07S,85,85,0
3,12910201MD307S,1291,0201MD307S,73,73,0
4,12910201N0A00S,1291,0201N0A00S,83,83,0
...,...,...,...,...,...,...
918,978N3A7M2H07S,978,N3A7M2H07S,46,46,0
919,978N3A7MD3EN,978,N3A7MD3EN,41,40,1
920,978N3A7N0A00S,978,N3A7N0A00S,45,45,0
921,978N3A7N1007S,978,N3A7N1007S,112,112,0


In [19]:
TD_STK = TD_STK.groupby('FILIAL')[['PRESENTE', 'STK_REAL', 'DIF_ABS']].sum().reset_index()
display(TD_STK)

,FILIAL,PRESENTE,STK_REAL,DIF_ABS
0,1291,2756,2681,83
1,1292,0,0,0
2,145,5310,5168,158
3,419,2790,2730,68
4,456,4512,3098,1428
5,59,2462,2308,160
6,978,5020,5142,338


In [20]:
TD_STK['Porcentagem%'] = (TD_STK['DIF_ABS'] / TD_STK['PRESENTE'])*100
TD_STK['Porcentagem%'] = TD_STK['Porcentagem%'].replace([np.inf, -np.inf], np.nan)
TD_STK['Porcentagem%'] = TD_STK['Porcentagem%'].fillna(0).round(0).astype(int)



In [21]:
TD_STK

,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%
0,1291,2756,2681,83,3
1,1292,0,0,0,0
2,145,5310,5168,158,3
3,419,2790,2730,68,2
4,456,4512,3098,1428,32
5,59,2462,2308,160,6
6,978,5020,5142,338,7


In [22]:
condicoes = [
    TD_STK["DIF_ABS"] == 0,
    TD_STK["Porcentagem%"].between(0,2, inclusive="both"),
    TD_STK["Porcentagem%"].between(3,5, inclusive="both"),
    TD_STK["Porcentagem%"].between(6,9, inclusive="both"),
    TD_STK["Porcentagem%"] >= 10,


]

liberado_choices = ['OK','OK','OK', 'OK', 'NÂO']
obs_choices = ['Sem Diferença', 'Até 2%', 'Até 5%', 'Abaixo de 10%', 'Analisar']

TD_STK['Liberado'] = np.select(condicoes, liberado_choices, default='Revisar Regra')
TD_STK['Obs'] = np.select(condicoes, obs_choices, default='Revisar Regra')

TD_STK['Obs2'] = ''

TD_STK





,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%,Liberado,Obs,Obs2
0,1291,2756,2681,83,3,OK,Até 5%,
1,1292,0,0,0,0,OK,Sem Diferença,
2,145,5310,5168,158,3,OK,Até 5%,
3,419,2790,2730,68,2,OK,Até 2%,
4,456,4512,3098,1428,32,NÂO,Analisar,
5,59,2462,2308,160,6,OK,Abaixo de 10%,
6,978,5020,5142,338,7,OK,Abaixo de 10%,


In [23]:
print("Iniciando a etapa 2: Reclassificação dos grupos 'Analisar'")

filiais_para_analisar = TD_STK.loc[TD_STK['Obs'] == 'Analisar', 'FILIAL'].unique()
filiais_para_analisar

Iniciando a etapa 2: Reclassificação dos grupos 'Analisar'


array(['456'], dtype=object)

In [24]:
TD_STK_analisar =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])['PRESENTE'].sum().reset_index()
TD_STK_analisar = TD_STK_analisar.merge(df_estoque[['chave', 'STK_REAL']], how='left', on='chave' )
TD_STK_analisar['STK_REAL'] = TD_STK_analisar['STK_REAL'].fillna(0).astype(np.int64)
TD_STK_analisar['DIF_ABS'] = (TD_STK_analisar['PRESENTE'] - TD_STK_analisar['STK_REAL']).abs()
TD_STK_analisar = TD_STK_analisar[TD_STK_analisar['FILIAL'].isin(filiais_para_analisar)]


TD_STK_analisar = TD_STK_analisar[TD_STK_analisar['DIF_ABS'] > 0]
TD_STK_analisar

,chave,FILIAL,ARTIGO_COR,PRESENTE,STK_REAL,DIF_ABS
512,4560111AX7EN,456,0111AX7EN,43,41,2
513,4560111M2H07S,456,0111M2H07S,33,30,3
514,4560111MD307S,456,0111MD307S,57,44,13
515,4560111N0A00S,456,0111N0A00S,67,35,32
518,456015MAXTEN,456,015MAXTEN,30,27,3
...,...,...,...,...,...,...
646,456N1TEN1007S,456,N1TEN1007S,60,53,7
647,456N3A7AX7EN,456,N3A7AX7EN,33,40,7
648,456N3A7M2H07S,456,N3A7M2H07S,35,32,3
649,456N3A7MD3EN,456,N3A7MD3EN,30,24,6


In [25]:
TD_STK_analisar.describe()

,PRESENTE,STK_REAL,DIF_ABS
count,87.000000,87.000000,87.000000
mean,35.436782,19.183908,16.413793
std,27.149329,20.455015,26.245953
min,4.000000,-28.000000,1.000000
25%,13.000000,5.000000,3.000000
50%,29.000000,14.000000,7.000000
75%,48.000000,31.500000,17.500000
max,127.000000,94.000000,143.000000


In [26]:
TD_STK_analisar.describe(include='object')

,chave,FILIAL,ARTIGO_COR
count,87,87,87
unique,87,1,87
top,4560111AX7EN,456,0111AX7EN
freq,1,87,1


In [28]:
def reavaliar_analisar(row):
    # Se já foi liberado na primeira etapa, mantém como está
    if row['Obs'] != 'Analisar':
        return row['Liberado'], row['Obs'], row['Obs2']

    filial_atual = row['FILIAL']
    
    # Busca os itens dessa filial no DataFrame detalhado
    # Ajuste 'TD_STK_analisar' se o nome do seu df detalhado for outro
    itens_filial = TD_STK_analisar[TD_STK_analisar['FILIAL'] == filial_atual]
    
    # Se não achar detalhes, retorna o que já estava
    if itens_filial.empty:
        return row['Liberado'], row['Obs'], "Sem detalhes encontrados"

    # Pega os 2 artigos com maior DIF_ABS
    top2_itens = itens_filial.sort_values(by='DIF_ABS', ascending=False).head(2)
    
    # Soma das diferenças dos 2 maiores
    soma_dif_top2 = top2_itens['DIF_ABS'].sum()
    artigos_ids = top2_itens['ARTIGO_COR'].tolist() # Para mensagem
    
    # Cálculo da NOVA Porcentagem (subtraindo os 2 maiores da diferença total)
    # Fórmula: (DIF_TOTAL - DIF_TOP2) / PRESENTE_TOTAL
    nova_dif = row['DIF_ABS'] - soma_dif_top2
    
    # Evita divisão por zero
    novo_perc = (nova_dif / row['PRESENTE'] * 100) if row['PRESENTE'] > 0 else 100
    novo_perc = round(novo_perc, 0) # Arredonda para facilitar comparação

    # --- LÓGICA DE DECISÃO ---
    
    # 1. Se baixou para 9% ou menos
    if novo_perc <= 9:
        return 'OK', 'Analisar', f"Liberar, exceto os artigos-cores {artigos_ids}"
    
    # 2. Se Presente > Stk Real E nova porcentagem entre 10% e 14%
    elif (row['PRESENTE'] > row['STK_REAL']) and (10 <= novo_perc <= 14):
        return 'OK', 'Analisar', f"Liberar, exceto os dois artigos cores, presente > stk Qlik {artigos_ids}"
    
    
    elif (row['PRESENTE'] < row['STK_REAL']):
        return 'NÃO', 'Analisar', "Muita diferença em vários artigos Presente < Stk Qlik"
    
    # 4. Caso residual (entre 10-14 mas Presente < Stk, ou entre 14-16, etc)
    else:
        return 'NÃO', 'Analisar', "Muita diferença em vários artigos"

# Aplica a função linha a linha (Apply)
# O zip agrupa os resultados para podermos atribuir a 3 colunas de uma vez
#Juntando as partes: TD_STK.apply(lambda row: ..., axis=1) significa:
#"Para cada linha (que chamaremos de row) no DataFrame TD_STK, execute o código que vem a seguir".
TD_STK[['Liberado', 'Obs', 'Obs2']] = TD_STK.apply(
    lambda row: pd.Series(reavaliar_analisar(row)), axis=1
)

# Exibe o resultado final
TD_STK

,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%,Liberado,Obs,Obs2
0,1291,2756,2681,83,3,OK,Até 5%,
1,1292,0,0,0,0,OK,Sem Diferença,
2,145,5310,5168,158,3,OK,Até 5%,
3,419,2790,2730,68,2,OK,Até 2%,
4,456,4512,3098,1428,32,NÃO,Analisar,Muita diferença em vários artigos
5,59,2462,2308,160,6,OK,Abaixo de 10%,
6,978,5020,5142,338,7,OK,Abaixo de 10%,


In [29]:
# Cria o agrupamento por 'Liberado' e 'Obs' e conta o tamanho (qtd de linhas)
resumo = TD_STK.groupby(['Liberado', 'Obs']).size().reset_index(name='Total')

# (Opcional) Ordena para ficar mais fácil de ler (Do maior para o menor)
resumo = resumo.sort_values(by='Total', ascending=True)

# Exibe o resumo
resumo



,Liberado,Obs,Total
0,NÃO,Analisar,1
2,OK,Até 2%,1
4,OK,Sem Diferença,1
1,OK,Abaixo de 10%,2
3,OK,Até 5%,2


In [30]:

total_filiais = resumo['Total'].sum()
print(f"Total de registros análise estoque {total_filiais}")

Total de registros análise estoque 7


In [31]:
# 1. Primeiro filtramos apenas as linhas onde Liberado é 'NÃO'
df_travado = TD_STK[TD_STK['Liberado'] == 'NÃO']

# 2. Agrupamos por Filial e Obs2, somando a coluna DIF_ABS
# O reset_index(name='Total') cria a coluna com o nome 'Total' igual ao seu print
resumo_travados = df_travado.groupby(['FILIAL', 'Obs2'])['DIF_ABS'].sum().reset_index(name='Total')

# (Opcional) Ordenar do maior para o menor erro total
resumo_travados = resumo_travados.sort_values(by='Total', ascending=False)

pd.set_option('display.max_colwidth', None)

# Agora exiba o resumo novamente
resumo_travados

,FILIAL,Obs2,Total
0,456,Muita diferença em vários artigos,1428


In [32]:
df_ressalvas = TD_STK[(TD_STK['Liberado'] == 'OK') & (TD_STK['Obs2'] != '')]

# 2. Agrupa por FILIAL e Obs2, somando o DIF_ABS
resumo_ressalvas = df_ressalvas.groupby(['FILIAL', 'Obs2'])['DIF_ABS'].sum().reset_index(name='Total')

# 3. Ordena do maior Total para o menor (opcional, ajuda a priorizar)
resumo_ressalvas = resumo_ressalvas.sort_values(by='Total', ascending=False)

# 4. Garante que o texto da Obs2 apareça inteiro
pd.set_option('display.max_colwidth', None)

# Exibe a tabela
resumo_ressalvas

,FILIAL,Obs2,Total


In [33]:
df_cat

,PontoVda,Sit,Usuario,Artigo,Pecas
0,145,ATENDIDO,WISE,0111AX7EN,18
1,1291,SUSPENSO COMERCIAL,WISE,0111AX7EN,6
2,978,SUSPENSO COMERCIAL,WISE,0111M2H07S,5
3,1291,SUSPENSO COMERCIAL,WISE,0111M2H07S,19
4,1291,SUSPENSO COMERCIAL,WISE,0111MD307S,30
...,...,...,...,...,...
659,1292,SUSPENSO FINANCEIRO,ABF,N3A7N0A00S,6
660,59,PENDENTE,ABF,N3A7N1007S,0
661,1292,SUSPENSO FINANCEIRO,ABF,N3A7N1007S,8
662,978,PENDENTE,ABF,N3A7NATEN,0


In [34]:
df_cat.dtypes

PontoVda     int64
Sit         object
Usuario     object
Artigo      object
Pecas        int64
dtype: object

In [35]:
df_cat.columns

Index(['PontoVda', 'Sit', 'Usuario', 'Artigo', 'Pecas'], dtype='object')

In [36]:
df_cat.columns = df_cat.columns.str.strip()

colunas_numericas = ['Pecas']
colunas_categoricas = ['Artigo', 'PontoVda', 'Sit', 'Usuario']

# --- AQUI ESTÁ A CORREÇÃO ---
# Usamos um loop caso você adicione mais colunas numéricas no futuro
for col in colunas_numericas:
    # Correção: to_numeric -> round(0) -> astype('Int64')
    df_cat[col] = pd.to_numeric(df_cat[col], errors='coerce').round(0).astype('Int64') 

# As categóricas continuam igual
df_cat[colunas_categoricas] = df_cat[colunas_categoricas].astype('str')

print("\n\n---- Depois da limpeza ---")
df_cat.dtypes



---- Depois da limpeza ---


PontoVda    object
Sit         object
Usuario     object
Artigo      object
Pecas        Int64
dtype: object

In [37]:
df_cat

,PontoVda,Sit,Usuario,Artigo,Pecas
0,145,ATENDIDO,WISE,0111AX7EN,18
1,1291,SUSPENSO COMERCIAL,WISE,0111AX7EN,6
2,978,SUSPENSO COMERCIAL,WISE,0111M2H07S,5
3,1291,SUSPENSO COMERCIAL,WISE,0111M2H07S,19
4,1291,SUSPENSO COMERCIAL,WISE,0111MD307S,30
...,...,...,...,...,...
659,1292,SUSPENSO FINANCEIRO,ABF,N3A7N0A00S,6
660,59,PENDENTE,ABF,N3A7N1007S,0
661,1292,SUSPENSO FINANCEIRO,ABF,N3A7N1007S,8
662,978,PENDENTE,ABF,N3A7NATEN,0


In [75]:
df_cat['chave'] = df_cat['PontoVda'] + df_cat['Artigo']
df_cat

C:\Users\gabriel.thiago\AppData\Local\Temp\ipykernel_15488\4117008666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['chave'] = df_cat['PontoVda'] + df_cat['Artigo']


,PontoVda,Sit,Usuario,Artigo,DtEmiNota,Pecas,chave
124,1292,ATENDIDO,WISE,0BEBNM2EN,,8,12920BEBNM2EN
127,1292,ATENDIDO,WISE,0BEDYVYEN,,13,12920BEDYVYEN
162,1292,PENDENTE,WISE,H1VW1CSN,,2,1292H1VW1CSN
172,1292,PENDENTE,WISE,H9UG1ASN,,10,1292H9UG1ASN
174,1292,PENDENTE,WISE,HCF91ASN,,2,1292HCF91ASN
175,1292,PENDENTE,WISE,HCF91BSN,,5,1292HCF91BSN
176,1292,PENDENTE,WISE,HCGM17SSN,,10,1292HCGM17SSN
218,1292,ATENDIDO,WISE,K6KN10TSI,,6,1292K6KN10TSI
262,1292,PENDENTE,WISE,KHZEN10SN,,12,1292KHZEN10SN
427,1292,SUSPENSO FINANCEIRO,ABF,0201AX7EN,,8,12920201AX7EN


In [39]:
df_cat["DtEmiNota"] = ''
# 1. Remove a coluna do final e guarda numa variável temporária
coluna_para_mover = df_cat.pop('DtEmiNota')

# 2. Insere ela na posição 4 (Logo após 'Usuario')
# A sintaxe é: .insert(posicao_numerica, 'nome_coluna', conteudo)
df_cat.insert(4, 'DtEmiNota', coluna_para_mover)

# Exibe o resultado
df_cat

C:\Users\gabriel.thiago\AppData\Local\Temp\ipykernel_15488\2238332843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat["DtEmiNota"] = ''


,PontoVda,Sit,Usuario,Artigo,DtEmiNota,Pecas,chave
124,1292,ATENDIDO,WISE,0BEBNM2EN,,8,12920BEBNM2EN
127,1292,ATENDIDO,WISE,0BEDYVYEN,,13,12920BEDYVYEN
162,1292,PENDENTE,WISE,H1VW1CSN,,2,1292H1VW1CSN
172,1292,PENDENTE,WISE,H9UG1ASN,,10,1292H9UG1ASN
174,1292,PENDENTE,WISE,HCF91ASN,,2,1292HCF91ASN
...,...,...,...,...,...,...,...
653,1292,SUSPENSO FINANCEIRO,ABF,N3A7M2H07S,,8,1292N3A7M2H07S
656,1292,SUSPENSO FINANCEIRO,ABF,N3A7MD3EN,,8,1292N3A7MD3EN
659,1292,SUSPENSO FINANCEIRO,ABF,N3A7N0A00S,,6,1292N3A7N0A00S
661,1292,SUSPENSO FINANCEIRO,ABF,N3A7N1007S,,8,1292N3A7N1007S


In [40]:
df_cat_fat_hoje_ontem

,PontoVda,Sit,Usuario,Artigo,DtEmiNota,Pecas
0,1291,FATURADO,WISE,0201A0LEN,2025-12-09,17
1,1291,FATURADO,WISE,0201AV7EN,2025-12-09,22
2,1291,FATURADO,WISE,0201H04EN,2025-12-09,24
3,1291,FATURADO,WISE,0201KRYEN,2025-12-09,20
4,1291,FATURADO,WISE,0201W0TEN,2025-12-09,23
...,...,...,...,...,...,...
124,419,FATURADO,SAP,KTNUEAMSI,2025-12-09,2
125,419,FATURADO,SAP,LZ82N10SI,2025-12-09,1
126,419,FATURADO,SAP,LZDL1BSI,2025-12-09,4
127,419,FATURADO,SAP,N3A7HMJEN,2025-12-09,8


In [41]:
df_cat_fat_hoje_ontem.columns

Index(['PontoVda', 'Sit', 'Usuario', 'Artigo', 'DtEmiNota', 'Pecas'], dtype='object')

In [42]:
df_cat_fat_hoje_ontem.columns = df_cat_fat_hoje_ontem.columns.str.strip()

colunas_numericas = ['Pecas']
colunas_categoricas = ['Artigo', 'DtEmiNota', 'PontoVda', 'Sit', 'Usuario']

# --- AQUI ESTÁ A CORREÇÃO ---
# Usamos um loop caso você adicione mais colunas numéricas no futuro
for col in colunas_numericas:
    # Correção: to_numeric -> round(0) -> astype('Int64')
    df_cat_fat_hoje_ontem[col] = pd.to_numeric(df_cat_fat_hoje_ontem[col], errors='coerce').round(0).astype('Int64') 

# As categóricas continuam igual
df_cat_fat_hoje_ontem[colunas_categoricas] = df_cat_fat_hoje_ontem[colunas_categoricas].astype('str')

print("\n\n---- Depois da limpeza ---")
df_cat_fat_hoje_ontem.dtypes



---- Depois da limpeza ---


PontoVda     object
Sit          object
Usuario      object
Artigo       object
DtEmiNota    object
Pecas         Int64
dtype: object

In [43]:
df_cat_fat_hoje_ontem['chave'] = df_cat_fat_hoje_ontem['PontoVda'] + df_cat_fat_hoje_ontem['Artigo']
df_cat_fat_hoje_ontem

,PontoVda,Sit,Usuario,Artigo,DtEmiNota,Pecas,chave
0,1291,FATURADO,WISE,0201A0LEN,2025-12-09,17,12910201A0LEN
1,1291,FATURADO,WISE,0201AV7EN,2025-12-09,22,12910201AV7EN
2,1291,FATURADO,WISE,0201H04EN,2025-12-09,24,12910201H04EN
3,1291,FATURADO,WISE,0201KRYEN,2025-12-09,20,12910201KRYEN
4,1291,FATURADO,WISE,0201W0TEN,2025-12-09,23,12910201W0TEN
...,...,...,...,...,...,...,...
124,419,FATURADO,SAP,KTNUEAMSI,2025-12-09,2,419KTNUEAMSI
125,419,FATURADO,SAP,LZ82N10SI,2025-12-09,1,419LZ82N10SI
126,419,FATURADO,SAP,LZDL1BSI,2025-12-09,4,419LZDL1BSI
127,419,FATURADO,SAP,N3A7HMJEN,2025-12-09,8,419N3A7HMJEN


In [44]:
df_cat_fat_hoje_ontem["DtEmiNota"] = ''
# 1. Remove a coluna do final e guarda numa variável temporária
coluna_para_mover = df_cat_fat_hoje_ontem.pop('DtEmiNota')

# 2. Insere ela na posição 1 (Logo após 'Artigo')
# A sintaxe é: .insert(posicao_numerica, 'nome_coluna', conteudo)
df_cat_fat_hoje_ontem.insert(4, 'DtEmiNota', coluna_para_mover)

# Exibe o resultado
df_cat_fat_hoje_ontem

,PontoVda,Sit,Usuario,Artigo,DtEmiNota,Pecas,chave
0,1291,FATURADO,WISE,0201A0LEN,,17,12910201A0LEN
1,1291,FATURADO,WISE,0201AV7EN,,22,12910201AV7EN
2,1291,FATURADO,WISE,0201H04EN,,24,12910201H04EN
3,1291,FATURADO,WISE,0201KRYEN,,20,12910201KRYEN
4,1291,FATURADO,WISE,0201W0TEN,,23,12910201W0TEN
...,...,...,...,...,...,...,...
124,419,FATURADO,SAP,KTNUEAMSI,,2,419KTNUEAMSI
125,419,FATURADO,SAP,LZ82N10SI,,1,419LZ82N10SI
126,419,FATURADO,SAP,LZDL1BSI,,4,419LZDL1BSI
127,419,FATURADO,SAP,N3A7HMJEN,,8,419N3A7HMJEN


In [45]:

df_consolidado = pd.concat([df_cat, df_cat_fat_hoje_ontem], ignore_index=True)

# Exibe o resultado final
df_consolidado

,PontoVda,Sit,Usuario,Artigo,DtEmiNota,Pecas,chave
0,1292,ATENDIDO,WISE,0BEBNM2EN,,8,12920BEBNM2EN
1,1292,ATENDIDO,WISE,0BEDYVYEN,,13,12920BEDYVYEN
2,1292,PENDENTE,WISE,H1VW1CSN,,2,1292H1VW1CSN
3,1292,PENDENTE,WISE,H9UG1ASN,,10,1292H9UG1ASN
4,1292,PENDENTE,WISE,HCF91ASN,,2,1292HCF91ASN
...,...,...,...,...,...,...,...
244,419,FATURADO,SAP,KTNUEAMSI,,2,419KTNUEAMSI
245,419,FATURADO,SAP,LZ82N10SI,,1,419LZ82N10SI
246,419,FATURADO,SAP,LZDL1BSI,,4,419LZDL1BSI
247,419,FATURADO,SAP,N3A7HMJEN,,8,419N3A7HMJEN


In [46]:
df_consolidado['Pecas'].sum()

np.int64(2408)

In [47]:
df_CA_agrupado = df_consolidado.groupby('chave')['Pecas'].sum().reset_index()
df_CA_agrupado
total = df_CA_agrupado['Pecas'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_CA_agrupado)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
             chave  Pecas
0    12910201A0LEN     17
1    12910201AV7EN     22
2    12910201H04EN     24
3    12910201KRYEN     20
4    12910201W0TEN     23
..             ...    ...
240     59KTVG3CSI     22
241     59KTVG3FSI     26
242     59LZ471BSI      8
243    59LZ47N0ASI      6
244     59LZ481ASI      4

[245 rows x 2 columns]

 O total de estoque real no BI é: 2408


In [48]:
TD_CA =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])[['RESERVADO', 'EMBALADO', 'TRANSITO']].sum().reset_index()
TD_CA = TD_CA.merge(df_CA_agrupado[['chave', 'Pecas']], how='left', on='chave' )
TD_CA['CA_BI'] = TD_CA['Pecas'].fillna(0).astype(np.int64)
TD_CA['DIF_ABS'] = (TD_CA['RESERVADO'] - TD_CA['CA_BI']).abs()
TD_CA = TD_CA.drop(columns=['Pecas'])
TD_CA

,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_BI,DIF_ABS
0,12910201AX7EN,1291,0201AX7EN,40,0,0,0,40
1,12910201HMJEN,1291,0201HMJEN,39,0,2,0,39
2,12910201M2H07S,1291,0201M2H07S,29,0,0,0,29
3,12910201MD307S,1291,0201MD307S,37,0,0,0,37
4,12910201N0A00S,1291,0201N0A00S,123,0,0,0,123
...,...,...,...,...,...,...,...,...
918,978N3A7M2H07S,978,N3A7M2H07S,10,0,0,0,10
919,978N3A7MD3EN,978,N3A7MD3EN,10,0,16,0,10
920,978N3A7N0A00S,978,N3A7N0A00S,14,0,3,0,14
921,978N3A7N1007S,978,N3A7N1007S,5,0,15,0,5


In [49]:
df_relatorio

,Cod. Loja,Artigo,Cor,Cancelada,Atendido (SAP)
0,100,0111,AX7EN,0,2
1,100,0111,AX7EN,0,2
2,100,0111,AX7EN,0,2
3,100,0111,M2H07S,0,2
4,100,0111,M2H07S,0,2
...,...,...,...,...,...
16203,87,N3A7,N1007S,0,7
16204,87,N3A7,NATEN,0,3
16205,87,N3A7,NATEN,0,12
16206,87,N3A7,NATEN,0,10


In [50]:
df_relatorio.columns

Index(['Cod. Loja', 'Artigo', 'Cor', 'Cancelada', 'Atendido (SAP)'], dtype='object')

In [51]:
df_relatorio.dtypes

Cod. Loja          int64
Artigo            object
Cor               object
Cancelada          int64
Atendido (SAP)     int64
dtype: object

In [52]:
df_relatorio['Cod. Loja'] = df_relatorio['Cod. Loja'].astype(str)

In [53]:
df_relatorio['chave'] = df_relatorio['Cod. Loja'] + df_relatorio['Artigo'] + df_relatorio['Cor']

In [54]:
df_relatorio = df_relatorio.groupby('chave')[['Atendido (SAP)', 'Cancelada']].sum().reset_index()
df_relatorio
total = df_relatorio['Atendido (SAP)'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_relatorio)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
              chave  Atendido (SAP)  Cancelada
0      1000111AX7EN               6          0
1     1000111M2H07S               5          0
2     1000111MD307S              11          0
3     1000111N0A00S               5          0
4      1000111NATEN               6          0
...             ...             ...        ...
6316    996KZHB1CSN               6          0
6317  996N1TEM2H07S               4          0
6318  996N1TEN0A00S               2          0
6319  996N1TEN1007S               2          0
6320  996N3A7N1007S               2          0

[6321 rows x 3 columns]

 O total de estoque real no BI é: 42858


In [55]:
TD_CA = TD_CA.merge(df_relatorio[['chave', 'Atendido (SAP)', 'Cancelada']], how='left', on='chave' )
TD_CA['Atendido (SAP)'] = TD_CA['Atendido (SAP)'].fillna(0).astype(np.int64)
TD_CA['Cancelada'] = TD_CA['Cancelada'].fillna(0).astype(np.int64)
TD_CA

,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_BI,DIF_ABS,Atendido (SAP),Cancelada
0,12910201AX7EN,1291,0201AX7EN,40,0,0,0,40,0,0
1,12910201HMJEN,1291,0201HMJEN,39,0,2,0,39,0,0
2,12910201M2H07S,1291,0201M2H07S,29,0,0,0,29,0,0
3,12910201MD307S,1291,0201MD307S,37,0,0,0,37,0,0
4,12910201N0A00S,1291,0201N0A00S,123,0,0,0,123,0,0
...,...,...,...,...,...,...,...,...,...,...
918,978N3A7M2H07S,978,N3A7M2H07S,10,0,0,0,10,0,0
919,978N3A7MD3EN,978,N3A7MD3EN,10,0,16,0,10,0,0
920,978N3A7N0A00S,978,N3A7N0A00S,14,0,3,0,14,0,0
921,978N3A7N1007S,978,N3A7N1007S,5,0,15,0,5,1,0


In [56]:
TD_CA['RESERVADO'].sum()

np.int64(7716)

In [57]:
TD_CA['CA_BI'].sum()

np.int64(1135)

In [58]:
TD_CA['Atendido (SAP)'].sum()

np.int64(1876)

In [59]:
TD_CA['DIF_ABS'].sum()

np.int64(6583)

In [60]:
TD_CA['Cancelada'].sum()

np.int64(0)

In [61]:
condicoes = [
    (TD_CA['RESERVADO'] > 0) & (TD_CA['CA_BI'] > 0),   # 1. Ambos maiores que 0
    (TD_CA['RESERVADO'] == 0) & (TD_CA['CA_BI'] > 0),  # 2. Reservado zerado, CA Qlik positivo
    (TD_CA['RESERVADO'] == 0) & (TD_CA['CA_BI'] == 0), # 3. Ambos zerados
    (TD_CA['RESERVADO'] > 0) & (TD_CA['CA_BI'] == 0)   # 4. Reservado positivo, CA Qlik zerado
]

# 2. Definimos os Cálculos Matemáticos para cada condição acima (na mesma ordem)
escolhas = [
    (TD_CA['RESERVADO'] - TD_CA['CA_BI']).abs(),       # 1. ABS(Reservado - Qlik)
    (TD_CA['CA_BI'] - TD_CA['TRANSITO']).abs(),        # 2. ABS(Qlik - Transito) -> Atenção aqui!
    TD_CA['RESERVADO'] - TD_CA['CA_BI'],               # 3. Reservado - Qlik
    TD_CA['RESERVADO'] - TD_CA['CA_BI']                # 4. Reservado - Qlik
]

# 3. Cria a coluna aplicando as regras
# O 'default=0' é caso alguma linha não caia em nenhuma regra (segurança)
TD_CA['dif_abs_trs'] = np.select(condicoes, escolhas, default=0)

# Exibe o resultado conferindo as colunas envolvidas
TD_CA[['RESERVADO', 'CA_BI', 'TRANSITO', 'dif_abs_trs']].head(10)

,RESERVADO,CA_BI,TRANSITO,dif_abs_trs
0,40,0,0,40
1,39,0,2,39
2,29,0,0,29
3,37,0,0,37
4,123,0,0,123
5,84,0,1,84
6,52,0,0,52
7,37,0,0,37
8,36,0,2,36
9,6,0,0,6


In [62]:
# A condição traduzida do Excel:
# E(J5<=K5; J5=F5)  --->  (Atendido <= Transito) & (Atendido == CA_QLIK)
condicao = (TD_CA['Atendido (SAP)'] <= TD_CA['TRANSITO']) & (TD_CA['Atendido (SAP)'] == TD_CA['CA_BI'])

# Aplica a lógica: np.where(condicao, valor_se_verdadeiro, valor_se_falso)
TD_CA['dif_aten_BI'] = np.where(
    condicao, 
    (TD_CA['Atendido (SAP)'] - TD_CA['CA_BI']).abs(),  # Se verdadeiro
    TD_CA['DIF_ABS']                                     # Senão (pega o valor que já existe na DIF_ABS)
)

# Exibe o resultado para conferência
TD_CA[['Atendido (SAP)', 'TRANSITO', 'CA_BI', 'DIF_ABS', 'dif_aten_BI']].head(10)

,Atendido (SAP),TRANSITO,CA_BI,DIF_ABS,dif_aten_BI
0,0,0,0,40,0
1,0,2,0,39,0
2,0,0,0,29,0
3,0,0,0,37,0
4,0,0,0,123,0
5,0,1,0,84,0
6,0,0,0,52,0
7,0,0,0,37,0
8,0,2,0,36,0
9,0,0,0,6,0


In [74]:
TD_CA 


Series([], Name: CA_BI, dtype: int64)

In [64]:
CA_agrupado = TD_CA.groupby('FILIAL')[['RESERVADO', 'CA_BI', 'EMBALADO', 'TRANSITO', 'DIF_ABS', 'Cancelada', 'dif_abs_trs', 'dif_aten_BI']].sum().reset_index()

CA_agrupado

,FILIAL,RESERVADO,CA_BI,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_BI
0,1291,2512,200,62,162,2312,0,2312,246
1,1292,842,836,0,0,8,0,8,8
2,145,3104,0,2774,357,3104,0,3104,0
3,419,0,0,0,0,0,0,0,0
4,456,4,0,0,1589,4,0,4,4
5,59,308,99,163,227,209,0,209,52
6,978,946,0,244,1712,946,0,946,5


In [65]:

condicoes_base = [
    CA_agrupado['DIF_ABS'] == 0,
    (CA_agrupado['DIF_ABS'] > 0) & (CA_agrupado['DIF_ABS'] < 11),
    (CA_agrupado['DIF_ABS'] >= 11) & (CA_agrupado['DIF_ABS'] <= 59)
]
escolhas_liberar = ['Ok', 'Ok', 'Ok']
escolhas_obs = ['Sem diferença', 'Até 10 peças de diferença', 'Pouca Diferença']

# Cria as colunas iniciais
CA_agrupado['Liberar'] = np.select(condicoes_base, escolhas_liberar, default='Analisar')
CA_agrupado['Obs'] = np.select(condicoes_base, escolhas_obs, default='Verificar Avançado')


print("Mapeando suspensões por filial...")

# 1. Filtra tudo que está suspenso no Qlik (df_consolidado)
df_suspensos = df_consolidado[
    df_consolidado['Sit'].str.startswith('SUSPENSO', na=False)
]

# 2. Cria um CONJUNTO DE PARCEIROS (Filial, Chave)
# Exemplo: O conjunto vai ter dados assim: {('657', '53C21NEN'), ('100', 'XPTO...')}
# Usamos .astype(str) para garantir que 657 (número) vire '657' (texto) e bata a comparação
chaves_suspensas_por_filial = set(
    zip(df_suspensos['PontoVda'].astype(str), df_suspensos['chave'])
)

# --- FUNÇÃO DE REVALIDAÇÃO ---'

def revalidar_carteira(row):
    # Se já foi liberado, passa reto
    if row['Liberar'] == 'Ok':
        return row['Liberar'], row['Obs']
    
    # Guarda a filial atual como texto para usar na comparação
    filial_atual_str = str(row['FILIAL'])
    
    # Pega os detalhes desta filial na tabela TD_CA
    detalhes = TD_CA[TD_CA['FILIAL'] == row['FILIAL']]
    
    if detalhes.empty:
        return 'NÃO', 'Sem detalhes na TD_CA'

    # --- REGRAS ANTERIORES (Mantidas) ---
    top2_itens = detalhes.sort_values(by='DIF_ABS', ascending=False).head(2)
    soma_dif_top2 = top2_itens['DIF_ABS'].sum()
    artigos_ids_top2 = top2_itens['ARTIGO_COR'].tolist()
    dif_restante_abs = row['DIF_ABS'] - soma_dif_top2
    
    if dif_restante_abs <= 59:
        return 'Ok', "Pouca Diferença"
    elif row['dif_abs_trs'] <= 59:
        return 'Ok', 'Pouca diferença (Regra Trânsito)'
    elif row['dif_aten_BI'] <= 59:
        return 'Ok', 'Pouca diferença (Regra Atendido)'

    top_item_aten = detalhes.sort_values(by='dif_aten_BI', ascending=False).head(2)
    soma_item_top2 = top_item_aten['dif_aten_BI'].sum()
    artigos_ids_aten = top_item_aten['ARTIGO_COR'].tolist()
    saldo_restante_aten = row['dif_aten_BI'] - soma_item_top2

    if saldo_restante_aten <= 59:
        return 'Ok', f"Liberar, exceto {artigos_ids_aten}"

    # --- SUA REGRA FINAL AJUSTADA (Cancelados + Suspensão Confirmada) ---
    else:
        # 1. Soma total de cancelados dessa filial
        total_cancelado = detalhes['Cancelada'].sum()
        
        # 2. Identifica quais chaves têm saldo no Qlik (CA_BI > 0)
        # (São esses itens que "sobram" e precisam estar justificados como suspensos)
        itens_com_saldo_qlik = detalhes[detalhes['CA_BI'] > 0]['chave'].tolist()
        
        # 3. VERIFICAÇÃO RIGOROSA:
        # "Para CADA item que tem saldo no Qlik, ele existe na lista de suspensos DESSA FILIAL?"
        todos_itens_estao_suspensos = True
        
        if len(itens_com_saldo_qlik) == 0:
            # Se não tem nenhum item com saldo Qlik, teoricamente não precisa checar suspensão
            todos_itens_estao_suspensos = False 
        else:
            for chave in itens_com_saldo_qlik:
                # O Pulo do Gato: Verifica se o par (Filial, Chave) existe no conjunto
                if (filial_atual_str, chave) not in chaves_suspensas_por_filial:
                    todos_itens_estao_suspensos = False
                    break # Se um falhar, já era, não é "Todos"
        
        # 4. APLICAÇÃO DA DECISÃO
        # Se cobriu o valor com cancelados E tudo que sobrou no Qlik está realmente suspenso nessa loja
        if (total_cancelado >= row['DIF_ABS']) and todos_itens_estao_suspensos:
            return 'NÃO', 'Todos os pedidos foram cancelados LW'
            
        else:
            return 'NÃO', 'Diferença Crítica'

    # --- EXECUÇÃO ---
print("Rodando validação com verificação de Filial+Chave...")

CA_agrupado[['Liberar', 'Obs']] = CA_agrupado.apply(
    lambda row: pd.Series(revalidar_carteira(row)), axis=1
)

# Exibe o resultado
CA_agrupado.head(6)

Mapeando suspensões por filial...
Rodando validação com verificação de Filial+Chave...


,FILIAL,RESERVADO,CA_BI,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_BI,Liberar,Obs
0,1291,2512,200,62,162,2312,0,2312,246,NÃO,Diferença Crítica
1,1292,842,836,0,0,8,0,8,8,Ok,Até 10 peças de diferença
2,145,3104,0,2774,357,3104,0,3104,0,Ok,Pouca diferença (Regra Atendido)
3,419,0,0,0,0,0,0,0,0,Ok,Sem diferença
4,456,4,0,0,1589,4,0,4,4,Ok,Até 10 peças de diferença
5,59,308,99,163,227,209,0,209,52,Ok,Pouca diferença (Regra Atendido)


In [66]:

# Tente assim se o anterior não funcionar
CARLOS = ['100', '331', '346', '1284']
CA_agrupado_filtrado = CA_agrupado[CA_agrupado['FILIAL'].isin(CARLOS)]

# Agora exibe
display(CA_agrupado_filtrado)

,FILIAL,RESERVADO,CA_BI,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_BI,Liberar,Obs


In [67]:
# Cria o agrupamento por 'Liberado' e 'Obs' e conta o tamanho (qtd de linhas)
resumo = CA_agrupado.groupby(['Liberar', 'Obs']).size().reset_index(name='Total')

# (Opcional) Ordena para ficar mais fácil de ler (Do maior para o menor)
resumo = resumo.sort_values(by='Total', ascending=False)

# O til (~) inverte a lógica: traz tudo que NÃO contém "exceto"
resumo_limpo = resumo[~resumo['Obs'].str.contains("exceto", case=False, na=False)]

print(resumo_limpo['Total'].sum())
# Exibe o resultado
resumo_limpo



7


,Liberar,Obs,Total
2,Ok,Pouca diferença (Regra Atendido),3
1,Ok,Até 10 peças de diferença,2
0,NÃO,Diferença Crítica,1
3,Ok,Sem diferença,1


In [68]:
# 2. Agrupamos e SOMAMOS as colunas numéricas
# ADICIONE O .reset_index() NO FINAL AQUI 👇
CA_OK = CA_agrupado.groupby(['FILIAL', 'Obs'])[['DIF_ABS', 'dif_abs_trs', 'dif_aten_BI']].sum().reset_index()

# Agora sim essa linha vai funcionar, pois 'Obs' voltou a ser uma coluna
resumo_OK = CA_OK[CA_OK['Obs'].str.contains("exceto", case=False, na=False)]

pd.set_option('display.max_rows', None)
resumo_OK

,FILIAL,Obs,DIF_ABS,dif_abs_trs,dif_aten_BI


In [69]:
# 1. Primeiro filtramos apenas as linhas onde Liberado é 'NÃO'
CA_travado = CA_agrupado[CA_agrupado['Liberar'] == 'NÃO']

# 2. Agrupamos por Filial e Obs2, somando a coluna DIF_ABS
# O reset_index(name='Total') cria a coluna com o nome 'Total' igual ao seu print
CA_travado = CA_travado.groupby(['FILIAL', 'Obs'])['DIF_ABS'].sum().reset_index(name='Total')

# (Opcional) Ordenar do maior para o menor erro total
resumo_travados = CA_travado.sort_values(by='Total', ascending=False)

pd.set_option('display.max_colwidth', None)

# Agora exiba o resumo novamente
resumo_travados

,FILIAL,Obs,Total
0,1291,Diferença Crítica,2312
